# Wavetable Synthesis and Envelopes
## George Tzanetakis 


Pitched musical sounds have a period component that results in the perception of a well-defined pitch. When viewed as a time-domain waveform it is clear that for most instrument sounds there is a short non-periodic attack portion of the sound that is critical for the perceived identity of the resulting sound. 

As memory costs when down in the 1980s, it became possible to store samples of musical sounds rather than generating them with analog circuitry. In order to save memory it is common to store just one loop instance of the periodic part of the sound and repeat it during playback as needed. The array storing the samples is called the **wavetable**. The attack portion can also be stored as an array digital samples. The resulting synthesizers are called samplers and were able to immitate acoustical musical instruments sounds much more accurately than analog synthesizers. 

The term **wavetable synthesis** is more general and refers to the use of stored arrays of samples as part of a synthesis architecture and not just using them directly for playback. 

Let's first take a look at how we can use this approach for sinusoidal playback. 

In [1]:
import numpy as np
import IPython.display as ipd


#plot_library = 'matplotlib'
plot_library = 'matplotlib_xkcd'
#plot_library = 'bokeh'


if (plot_library=='bokeh'):
    import bokeh 
    from bokeh.io import output_notebook
    from bokeh.plotting import figure, output_file, show
    output_notebook()

    def plot(data_list): 
        p = figure(plot_height=300, plot_width=600, title='Synthesizers')
        for data in data_list: 
            p.line(np.arange(0,len(data)), data)
        show(p)
        
if (plot_library=='matplotlib'): 
    %matplotlib notebook 
    import matplotlib.pyplot as plt
    def plot(data_list,label_list=[],xlabel='', ylabel='', title=''):
        fig, ax = plt.subplots(figsize=(8,4))
        for (data,label) in zip(data_list, label_list): 
            plt.title('Synth-CS: '+title)
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.plot(np.arange(0, len(data)), data, label=label)
        if (label_list):
            ax.legend()
        
        
if (plot_library=='matplotlib_xkcd'): 
    %matplotlib notebook 
    import matplotlib.pyplot as plt

    def plot(data_list,label_list=[],xlabel='', ylabel='', title=''):
        fig, ax = plt.subplots(figsize=(8,4))  
        plt.xkcd()
        if not(label_list):
            for d in enumerate(data_list): 
                label_list.append('')
        for (data,label) in zip(data_list, label_list): 
            plt.title('Synth-CS: '+title)
            plt.xlabel(xlabel)
            plt.ylabel(ylabel)
            plt.plot(np.arange(0, len(data)), data, label=label)  
            ax.legend()
    

In [ ]:
# generate a discrete time sine signal with a specified amplitude, frequency, duration, and 
# phase 
def sinusoid(freq=440.0, dur=1.0, srate=44100.0, amp=1.0, phase = 0.0): 
    t = np.linspace(0,dur,int(srate*dur))
    data = amp * np.sin(2*np.pi*freq *t+phase)
    return data

srate = 48000
data = sinusoid(freq=261.63, dur=2, srate=srate)
plot([data[0:1000]], ['sin(t)'], 'Samples', 'Amplitude', 'Sinusoid at 440Hz')
ipd.Audio(data, rate=srate)

Using this approach we are able to create sinusoidal sounds with user-specified frequency and duration which means we can create simple melodies and potentially polyphony by appropriately summing the generated audio signals. 

However, there are several limitations of this approach. Every audio sample is computed by calling the np.sin() function. The longer the duration of the generated sound, the more calls need to be made. We know that the **sin** function is periodic so it should be possible to only compute the values of the function for one period and then reuse them. 
Another limitation is that we can only generate sinusoidal audio signals - ideally we would like a more general approach that can accomodate the generation of any periodic signal. We also would like to change (modulate) dynamically the frequency of the oscillator to generate more interesting signals. For example, analog synthesizers frequently provide a Low Frequency Oscillator (LFO) that can be use to modulate the frequency of a Voltage Control Oscillator (VCO). LFOs change at slow rates to provide effects like vibrato. 

Let's try to modify the code above by adding a LFO signal. 

In [ ]:
def sinusoid_lfo_attempt1(freq = 440.0, dur=1.0, srate = 48000, amp=1.0, 
                          mode = 'lfo'):
        t = np.arange(0,int(srate*dur))
        lfo = np.sin(2 * np.pi * 2/srate * t)
        if mode == 'lfo': 
            f = freq + 100 * lfo  # f is an array 
        elif mode == 'constant': 
            f = freq  # f is a single value 
        else: 
            f = freq
            print('Unsupported mode')
        data = amp * np.sin(2*np.pi*f/srate * t)       
        return data
    
data = sinusoid_lfo_attempt1(freq=261.63, dur=3.0, mode='lfo')   # C4
plot([data[0:3000]], ['LFO sin(t)'], 'Samples', 'Amplitude', 'Failed LFO')
ipd.Audio(data, rate=srate)

The resulting sound is interesting but not what we would expect for an LFO-modulated sinusoid. Instead of a a vibrato around a fixed frequency it keeps increasing in frequency. 
This has to do with the difference between instanteneous frequency and phase and frequency of the sinusoid. For those of you interested in the more signal processing aspects you can read more about this here: https://en.wikipedia.org/wiki/Instantaneous_phase_and_frequency

To fix this problem we need to calculate how much to increase the phase at every sample. To make this process more explicit we will not take advantage of the vector operation of numpy and instead have an explicit loop. This would also be how one would code this in a lower level language like C or Assembly. 

In [ ]:
def sinusoid_lfo_loop1(freq = 440.0, dur = 1.0, srate = 48000, amp=1.0, 
                      apply_lfo=True): 
    # allocate arrays 
    indices = np.arange(0, int(srate*dur))
    f = np.zeros(indices.shape)
    p = np.zeros(indices.shape)
    lfo = np.zeros(indices.shape)
    data = np.zeros(indices.shape)
    
    # loop version 
    for i in indices: 
        lfo[i] = np.sin(2 * np.pi * 2 / srate *i)
        if apply_lfo: 
            f[i] = freq + 100.0 * lfo[i]
        else: 
            f[i] = freq   
        if (i > 0): 
            p[i] = (p[i-1] + f[i]/srate)  
        data[i] = amp * np.sin(2 * np.pi * p[i])
    plot([data[0:2000], p[0:2000]], ['data', 'phase'])
    return data 

data = sinusoid_lfo_loop1(freq=200.63, dur = 4.0, apply_lfo=True)
ipd.Audio(data, rate=srate)



Notice that the phase keeps increasing. This is not ideal because over longer time periods it could be come really big and cause numerical problems. Because the sinusoidal signal is periodic we can reset the phase every period using a mod operation. The resulting signal sounds the same but observe how the plot changes. 

In [ ]:
def sinusoid_lfo_loop2(freq = 440.0, dur = 1.0, srate = 48000, amp=1.0, 
                      apply_lfo=True, lfo_freq=2.0, lfo_range=100.0): 
    # allocate arrays 
    indices = np.arange(0, int(srate*dur))
    f = np.zeros(indices.shape)
    p = np.zeros(indices.shape)
    lfo = np.zeros(indices.shape)
    data = np.zeros(indices.shape)
    
    # loop version 
    for i in indices: 
        lfo[i] = np.sin(2 * np.pi * lfo_freq / srate *i)
        if apply_lfo: 
            f[i] = freq + lfo_range * lfo[i]
        else: 
            f[i] = freq   
        if (i > 0): 
            # here is the mod operation - the resulting phase increment 
            # has a saw tooh patten 
            p[i] = (p[i-1] + f[i]/srate) % 1.0 
        data[i] = amp * np.sin(2 * np.pi * p[i])
    plot([data[0:1000], p[0:1000]])
    return data 

data = sinusoid_lfo_loop2(freq=261.63, dur = 4.0, apply_lfo=True)
ipd.Audio(data, rate=srate)



**Sidenote**: If we increase the frequency of the LFO instead of hearing a vibrato, we start affecting the timbre of the resulting sound. This is the basis of **Frequency Modulation** synthesis technique we will explore later. Notice that the phase follows a distorted sawtooth pattern. 

In [ ]:
data = sinusoid_lfo_loop2(freq=261.63, dur = 4.0, apply_lfo=True, lfo_freq = 100, lfo_range=100)
ipd.Audio(data, rate=srate)

Now that we better understand the phase increment, we can recreate the above function in vector form. Notice that the np.cumsum keeps increasing in the code below. We could write a version that does the mod operation so that the phase would reset in a saw tooh fashion. This is left as an activity for the reader. 

In [ ]:
def sinusoid_lfo_vectorized(freq = 440.0, dur = 1.0, srate = 48000, amp=1.0, apply_lfo=True, lfo_freq=2.0, lfo_range=100): 
    indices = np.arange(0, int(srate*dur))
    f = np.zeros(indices.shape)
    p = np.zeros(indices.shape)
    lfo = np.zeros(indices.shape)
    data = np.zeros(indices.shape)
    
    lfo = np.sin(2 * np.pi * lfo_freq / srate *indices)
    if apply_lfo: 
        f = freq + lfo_range * lfo
    else: 
        f = np.repeat(freq,indices.shape[0])
    p = np.cumsum(f/srate)
    data = amp * np.sin(2 * np.pi * p)
    plot([data[0:1000], p[0:1000]])
    return data 

data = sinusoid_lfo_vectorized(freq=261.63, dur = 2.0, apply_lfo=True)
ipd.Audio(data, rate=srate)


Now that we have dealt with the issue of instantaneous frequency and phase let's look at reducing the memory footprint of the calculation. We know that the underlying sinusoidal signal is periodic. The idea behind a wavetable is to calculate and store the samples of one period of the waveform and then reuse the computed sampels as needed. 

Let's try to see if we can create a sound by only computing one period of the underlying periodic function and then repeating it. Notice that I have "cooked" the numbers so that the resulting wavetable size is an integer number of samples. This is typically not the case and we will look at how to deal with it later in this notebook. 


In [ ]:
srate = 16000 
freq = 160 
wavetable_size = srate / freq
print('Wavetable size', wavetable_size)

t = np.arange(0, wavetable_size)
wavetable = np.sin(2 * np.pi * freq / srate * t)
plot([wavetable])

# repeat/tile the wavetable 320 times to create the data array 
data = np.tile(wavetable, 320)
ipd.Audio(data, rate=srate)

Let's see if we can make this approach a little bit more general. 
The wavetable is created for a specific frequency and sample rate and then reused to a generate a sound of user-specified duration. This saves both computation (the np.sin calls are only done once at the start when **create_wavetable** is called) and memory. Notice that we still allocate the fully array of samples (data) for playback but in real-time scenarios we would generate the samples in small buffers as we will see later. 

One small thing to notice is that the true period for the wavetable is 
srate / frequency which is 183.46 samples. This is rounded up to 184 which means that the wavetable contains a little bit more than one period. This can cause issues when it is repeated. 

There are still some limitations with this approach. The wavetable needs to be recreated any time we want to play a sound of different frequency and also only supports the playback of sinusoids. We will address these limitations below. 


In [ ]:
# create a wavetable holding the samples of one period of the waveform 

def create_wavetable(freq, srate):
    # the wavetable length will be the period corresponding
    # to the frequency at the specified sample rate
    wavetable_length = int(srate / freq) 
    wavetable = np.zeros(wavetable_length)
    f = np.zeros(wavetable_length)
    p = np.zeros(wavetable_length)
    for i in np.arange(0,wavetable_length): 
        f[i] = freq
        if (i>0): 
            p[i] = (p[i-1]+f[i]/srate) % 1.0 
        wavetable[i] = np.sin(2 * np.pi * p[i])

    plot([wavetable[0:wavetable_length], p[0:wavetable_length]])
    return wavetable 

# generate a user-specified duration of a sinusoidal signal 
# using a wavetable

def sinusoid_wavetable(freq = 440.0, dur = 1.0, srate = 48000, 
                       amp=1.0):    
    wavetable = create_wavetable(freq, srate)
    indices = np.arange(0, int(srate*dur))
    data = np.zeros(indices.shape)

    for i in indices: 
        wavetable_index = i % wavetable.shape[0] 
        # read the samples from the wavetable 
        data[i] = wavetable[wavetable_index] 
    plot([data[0:1000]])
    return data 

data = sinusoid_wavetable(freq=440, dur = 2.0)
ipd.Audio(data, rate=srate)



We can generalize the notion of a wavetable to compute any function by storing samples of that function in memory. We can define different function for different types of periodic functions that are common in synthesizers such as sawtooh, square, and triangle waveforms. We will also like to only have one wavetable and be able to generate any frequency from it. This can be done by computing different phase increments depending on the frequency rather than reading from the wavetable one value at a time. Another way to view this is as reading from the wave table at different sampling rates. 

In order, to return a value for non-integer phase increments some form of interpolation is needed. The **wavetable_lookup** function serves this purposes and supports three different approaches. 

In [ ]:
# let's define some basic periodic waveforms
def sawtooth(x):
    return (x - np.pi)/np.pi

def square(x):
    return np.sign(sawtooth(x))

def triangle(x):
    return 1 - 2 * np.abs(sawtooth(x))


# notice that the wavetable just stores 
# values of the function fn 
def create_wavetable(length, fn):
    L = length+1 # add one sample to wavetable to allow edge interpolation
    t = np.linspace(0, 1.0 , L)
    wavetable = fn(2 * np.pi * t)
    return wavetable
    
# wavetable look (truncate and round only use one sample), 
# interpolate uses two samples and is more accurate 
def wavetable_lookup(phase_index, wavetable, mode): 
    if (mode == 'truncate'): 
        return wavetable[int(phase_index)]
    elif (mode == 'round'): 
        return wavetable[round(phase_index)]
    elif (mode == 'interpolate'): 
        x  = phase_index
        x0 = int(phase_index)
        x1 = x0+1
        y0 = wavetable[x0]
        y1 = wavetable[x1]
        return y0 * (x1-x) + y1 * (x - x0)
    else:
        return 0.0 
    

In [ ]:
table_length = 1000
phase_index = 0 
freq = 220 
phase_increment = (table_length * freq) / srate 
data = np.zeros(srate)

wavetable = create_wavetable(table_length, np.sin)
plot([wavetable])
for t in np.arange(0,srate): 
    phase_index = (phase_index + phase_increment) % table_length
    data[t] = wavetable_lookup(phase_index, wavetable, mode='round')
ipd.Audio(data, rate=srate)

Let's try a different waveform 

In [ ]:
wavetable = create_wavetable(table_length, triangle)
duration_samples = 2 * srate 
samples = np.arange(0, duration_samples)
data = np.zeros(duration_samples)

freq = 220 
phase_increment = (table_length * freq) / srate 
for s in samples: 
    phase_index = (phase_index + phase_increment) % table_length
    data[s] = wavetable_lookup(phase_index, wavetable, mode='round')

plot([data[0:2000]])   
ipd.Audio(data, rate=srate)

Notice that we we can use a LFO to modify the phase increment for every sample and get the right effect with any periodic waveform. As you can see from the plots the low frequency changes in the frequency results in corresponding changes in the phase increment reading from the wavetable. 
This process is called **frequency modulation**. 

In [ ]:
wavetable = create_wavetable(table_length, triangle)
lfo = np.sin(2 * np.pi * 100 / srate * samples)
f = 220 + 100.0 * lfo
phase_increment = (table_length * f) / srate 

for s in samples: 
    phase_index = (phase_index + phase_increment[s]) % table_length
    data[s] = wavetable_lookup(phase_index, wavetable, mode='round')

plot([data[0:1000]])
plot([f[0:duration_samples]])
plot([phase_increment[0:duration_samples]])
ipd.Audio(data, rate=srate)


We can also use **frequency modulation** to create other effects like the 
exponential frequency sweep below. 

In [ ]:
wavetable = create_wavetable(table_length, triangle)

base = 1000
f = (base**np.linspace(0, 1, duration_samples) - 1) / (base - 1) * 3500 + 500
phase_increment = (table_length * f) / srate 

for s in samples: 
    phase_index = (phase_index + phase_increment[s]) % table_length
    data[s] = wavetable_lookup(phase_index, wavetable, mode='round')

plot([data[0:1000]])
plot([f[0:duration_samples]])
plot([phase_increment[0:duration_samples]])
ipd.Audio(data, rate=srate)



Alternatively, we can use a LFO to change the amplitude of the resulting signal. This is called **amplitude modulation**. 


In [ ]:
wavetable = create_wavetable(table_length, triangle)
lfo = np.sin(2 * np.pi * 3 / srate * samples)
a = 0.5 + 0.4 * lfo
phase_increment = (table_length * freq) / srate 
for s in samples: 
    phase_index = (phase_index + phase_increment) % table_length
    data[s] = a[s] * wavetable_lookup(phase_index, wavetable, mode='round')

plot([data[0:duration_samples]])
ipd.Audio(data, rate=srate)

If we make the wavetable too small then we start hearing artifacts because of the quantization and interpolation. 

In [ ]:
table_length = 20
phase_index = 0 
freq = 220 
phase_increment = (table_length * freq) / srate 
data = np.zeros(srate)

wavetable = create_wavetable(table_length, np.sin)
for t in np.arange(0,srate): 
    phase_index = (phase_index + phase_increment) % table_length
    data[t] = wavetable_lookup(phase_index, wavetable, mode='round')
plot([data[0:1000]])
ipd.Audio(data, rate=srate)

We can also create reasonable imitations of some musical instruments by sampling their time domain waveform and carefully selecting the repeating pattern as the wavetable. Here is an example with a trumpet sound. Notice the importance of the attack that is missing from the looped sample. A simple exercise for the reader is to isolate the attack of the sampled trumpet sound and play it before looping the wavetable for a more realistic sound. 

In [ ]:
import librosa
(y,srate) = librosa.load('trumpet_loop_48223.wav', sr=44100, mono=True)
plot([np.tile(y,3),y])
phase_index = 0 
phase_increment = 1 
data = np.zeros(srate)
wavetable = y
table_length = y.shape[0]
for t in np.arange(0,srate): 
    phase_index = (phase_index + phase_increment) % table_length
    data[t] = wavetable_lookup(phase_index, wavetable, mode='round')
ipd.Audio(data, rate=srate)

In [ ]:
# trumpet sound from freesound 
(y,srate) = librosa.load('48223__slothrop__trumpetc2.wav', sr=44100, mono=True)
plot([y[600:1600]])
ipd.Audio(y, rate=srate)

Any set of value that can be repeated periodically without discontinuities will result in a pitched sound. For example you can create various pitched sounds using a random polynomial as the wavetable. 

In [ ]:
L = 256
P = 14
# generate random polynomial control points
x = np.linspace(0, L, P)
y = np.hstack([0, np.random.normal(size=P - 2), 0])
# fit the random polynomial
p = np.polyfit(x, y, deg=7)

# generate wavetable 
wavetable = np.polyval(p, np.arange(L))
wavetable /= np.max(np.abs(wavetable))
srate = 44100
plot([np.tile(wavetable,3),wavetable])
phase_index = 0 
phase_increment = 1 
data = np.zeros(srate)
table_length = wavetable.shape[0]
for t in np.arange(0,srate): 
    phase_index = (phase_index + phase_increment) % table_length
    data[t] = wavetable_lookup(phase_index, wavetable, mode='round')
ipd.Audio(data, rate=srate)

One can analyze the error of different types of interpolation 
when using table lookup for sinusoidal singnals: 

Moore FR. Table lookup noise for sinusoidal digital oscillators. Computer Music Journal. 1977 Apr 1:26-9.
https://www.jstor.org/stable/pdf/23320138.pdf

As well as generalize to more than sinusoids: 
Dannenberg RB. Interpolation error in waveform table lookup. International Computer Music Conference, 1998 

https://kilthub.cmu.edu/articles/Interpolation_Error_in_Waveform_Table_Lookup/6606596/files/12097115.pdf 



It is straightforward to extend the code above in various ways to enable more possibilities. Here are some examples: 

1. The sawtooth signals that correspond to the phase increments are sometimes called phasors. One can support polyphony using the same wavetable by using multiple phasors to generate different frequencies. One can think of the wavetable as a a circular array of values sometimes called a ring buffer that is being read at different sampling rates using interpolation. It is also possible to have multiple wavetables for supporting polyphony. The multiple phasors approach supports polyphony of the same underlying waveform but having multiple wavetables supports polyphony with potentially different oscillators (multi-timbrality) 

2. To provide more flexibility the wavetable oscillator(s) can take as argument time-varying envelopes for amplitude, frequency, and phase. For example one could add Attack, Decay, Sustain, Release envelopes for the amplitude. This generalizes oscillators to the concept of unit-generators that are fundamental building blocks in most computer music languages and programming environments and enables various types of interesting modulations. 

3. In languages that support object-oriented programming one can write the oscillators as objects and keep their state (for example phase-increment) as part of the object. This also allows the creation of more complex architectures like patches of unit generators. 

